In [1]:
pip install nltk

In [2]:
import re
import io
import string
import nltk
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\momen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# I used googlecolab because it is faster than my laptop
# from google.colab import files
# uploaded = files.upload()

In [7]:
df=pd.read_csv('emails.csv')
df = df.sample(frac = 1)

df.head()

,text,spam
678,"Subject: important : $ 97438 dear homeowner ,...",1
4255,Subject: prof . at rice here is this prof ' s...,0
3809,Subject: re : fw : wharton resume submission -...,0
3637,Subject: summer associate mentor the summer a...,0
2480,Subject: re : request for two powerpoint prese...,0


In [8]:
df.shape

(5728, 2)

In [12]:
#Reduce the size of dataset since my pc memory is not enough.
df= df.iloc[1000::]
df.shape

(1728, 2)

In [13]:
# Eliminating stop words and clean data
def text_proc(txt):
    punct_less = [char for char in txt if char not in string.punctuation]
    punct_less = ''.join(punct_less)
    final_txt = [word for word in punct_less.split() if word.lower() not in stopwords.words('english')]
    return ' '.join(final_txt)

In [14]:
def preprocessing_data(df):
    df["text"] = df["text"].apply(lambda x :text_proc(x)) 
    return df

In [15]:
def dataframe_to_list(train,test):
    corpus = train['text'].tolist()
    test = test['text'].tolist()
    return corpus, test

In [16]:
def creating_ngram_model(train_corpus, text_corpus,gram):
    vectorizer  = CountVectorizer(ngram_range = (gram,gram))
    counts = vectorizer.fit_transform(corpus)
    
    test_cv = CountVectorizer(ngram_range = (gram,gram))
    test_counts = test_cv.fit_transform(test_corpus)
    return vectorizer,counts,test_cv, test_counts

In [17]:
def counter(train):
    return len(train.loc[train['spam'] == 1]), len(train.loc[train['spam'] == 0])  

In [18]:
def create_matixes(train,counts):
    matrix = counts.toarray()
    spams = [train.iloc[i, 1] == 1  for i in range(len(train))]
    hams = [train.iloc[i, 1] == 0 for i in range(len(train))]
    return matrix[spams], matrix[hams]

In [19]:
def occurence(spamOccurence,hamOccurence,features):
    occurenceDict = {}
    for i in range(len(features)): 
        occurenceDict[features[i]] = {'spam': spamOccurence[i], 'ham': hamOccurence[i], 'diff': hamOccurence[i] - spamOccurence[i], 'total': hamOccurence[i] + spamOccurence[i]}  
    return pd.DataFrame.from_dict(occurenceDict, orient='index')
    
def ham_spam_occurence(spamFilter, hamFilter):
    spamOccurence=  spamFilter.sum(axis = 0)
    hamOccurence = hamFilter.sum(axis = 0)
    return spamOccurence, hamOccurence
    

In [20]:
df = preprocessing_data(df)
train, test = train_test_split(df, test_size=0.2)
corpus, test_corpus = dataframe_to_list(train, test)

In [21]:
# Create unigram
vectorizer,counts,test_cv,test_counts =  creating_ngram_model(corpus,test_corpus,1)

In [22]:
spamCount,hamCount = counter(train)

In [23]:
# Create  2 matrix for spams and hams to use them in naive_bayes algorithm
spamFilter, hamFilter = create_matixes(train,counts)

In [24]:
spamOccurence ,hamOccurence = ham_spam_occurence(spamFilter, hamFilter)
features = vectorizer.get_feature_names()
occurenceDf = occurence(spamOccurence,hamOccurence,features)

In [25]:
occurenceDf.sort_values('spam').tail(10)

,spam,ham,diff,total
http,140,194,54,334
get,149,327,178,476
information,150,394,244,544
adobe,152,6,-146,158
company,154,230,76,384
email,156,269,113,425
money,166,19,-147,185
business,200,286,86,486
com,236,961,725,1197
subject,386,2047,1661,2433


In [26]:
occurenceDf.sort_values('diff').tail(10)

,spam,ham,diff,total
please,118,984,866,1102
would,59,927,868,986
cc,0,914,914,914
hou,0,1122,1122,1122
kaminski,0,1164,1164,1164
2000,2,1292,1290,1294
subject,386,2047,1661,2433
vince,1,2072,2071,2073
ect,1,2337,2336,2338
enron,0,3004,3004,3004


In [27]:
occurenceDf.sort_values('ham').tail(10)

,spam,ham,diff,total
would,59,927,868,986
com,236,961,725,1197
please,118,984,866,1102
hou,0,1122,1122,1122
kaminski,0,1164,1164,1164
2000,2,1292,1290,1294
subject,386,2047,1661,2433
vince,1,2072,2071,2073
ect,1,2337,2336,2338
enron,0,3004,3004,3004


In [28]:
def getLaplaceSmoothedProb(occurenceDf,totalTokenCount, word, classification, totalSpam,totalHam):
    if word in occurenceDf.index:
        if (classification == 'spam'):
            return np.log((1 + occurenceDf.loc[word].spam) / (totalSpam + totalTokenCount))
        else:
            return np.log((1 + occurenceDf.loc[word].ham) / (totalHam + totalTokenCount))
    else:
        if (classification == 'spam'):
            return np.log(1 / (totalSpam + totalTokenCount))
        else:
            return np.log(1 / (totalHam + totalTokenCount))

In [29]:
def naive_bayes(occurenceDf, spamCount,hamCount,test_features,test_rows,totalSpam,totalHam ):
    TP, FP, FN, TN = 0,0,0,0
    totalTokenCount = len(occurenceDf)
    spamLogProb = np.log(spamCount / (spamCount + hamCount))
    hamLogProb = np.log(hamCount / (spamCount + hamCount))
    
    for i in range(len(test_rows)):
        row = test_rows[i]
        test_dict = {test_features[index]: row[index] for index in range(len(row)) if row[index] != 0}
        spamProb = spamLogProb
        hamProb = hamLogProb
        for item in test_dict:
            spamProb += (getLaplaceSmoothedProb(occurenceDf,totalTokenCount, item, 'spam',totalSpam,totalHam) * test_dict[item])
            hamProb += (getLaplaceSmoothedProb(occurenceDf,totalTokenCount,  item, 'ham',totalSpam,totalHam) * test_dict[item])
        if (spamProb > hamProb and test.iloc[i,1] == 1) or (hamProb > spamProb and test.iloc[i,1] == 0):
            if test.iloc[i,1] == 1:
                TN +=1
            else:
                TP +=1
        else:
            if test.iloc[i,1] == 1:
                FN +=1
            else:
                FP +=1
    return  TP, FP, FN, TN

In [30]:
test_features = test_cv.get_feature_names()
test_rows = test_counts.toarray()
totalSpam = occurenceDf['spam'].sum()
totalHam = occurenceDf['ham'].sum()

In [31]:
TP, FP, FN, TN = naive_bayes(occurenceDf, spamCount,hamCount,test_features,test_rows, totalSpam,totalHam)

In [32]:
accuracy = (TP + TN) / (TP +  FP+ TN + FN)
precision = TP /(TP +FP)
recall = TP /(TP +FN)
F1Score = (2*(recall* precision))/(precision  +recall )

In [33]:
print("Accuracy is {:.2f} %".format(accuracy))

Accuracy is 0.99 %


In [34]:
print("Precision is {:.2f} %".format(precision))

Precision is 0.98 %


In [35]:
print("Recall is {:.2f} %".format(recall))

Recall is 1.00 %


In [36]:
print("F1Score is {:.2f} %".format(F1Score))

F1Score is 0.99 %


# Without stop words

In [37]:
df=pd.read_csv('emails.csv')

In [38]:
df = df.sample(frac = 1)
df= df.iloc[1000::]
df.shape


(4728, 2)

In [48]:
train, test = train_test_split(df, test_size=0.2)
corpus, test_corpus = dataframe_to_list(train, test)
# Create unigram
vectorizer,counts,test_cv,test_counts =  creating_ngram_model(corpus,test_corpus,1)

In [49]:
spamCount,hamCount = counter(train)
spamFilter, hamFilter = create_matixes(train,counts)
spamOccurence ,hamOccurence = ham_spam_occurence(spamFilter, hamFilter)
features = vectorizer.get_feature_names()
occurenceDf = occurence(spamOccurence,hamOccurence,features)

MemoryError: Unable to allocate 295. MiB for an array with shape (1634, 23691) and data type int64

In [ ]:
occurenceDf.sort_values('spam').tail(10)

In [ ]:
occurenceDf.sort_values('diff').tail(10)

In [50]:
occurenceDf.sort_values('ham').tail(10)

,spam,ham,diff,total
please,103,1039,936,1142
would,57,1067,1010,1124
com,186,1207,1021,1393
kaminski,0,1226,1226,1226
2000,9,1276,1267,1285
hou,8,1498,1490,1506
subject,369,2109,1740,2478
vince,0,2153,2153,2153
ect,0,3044,3044,3044
enron,0,3414,3414,3414


In [51]:
test_features = test_cv.get_feature_names()
test_rows = test_counts.toarray()
totalSpam = occurenceDf['spam'].sum()
totalHam = occurenceDf['ham'].sum()
TP, FP, FN, TN = naive_bayes(occurenceDf, spamCount,hamCount,test_features,test_rows, totalSpam,totalHam)

In [52]:
accuracy = (TP + TN) / (TP +  FP+ TN + FN)
precision = TP /(TP +FP)
recall = TP /(TP +FN)
F1Score = (2*(recall* precision))/(precision  +recall )


In [53]:
print("Accuracy is {:.3f} %".format(accuracy))

Accuracy is 0.826 %


In [54]:
print("Precision is {:.2f} %".format(precision))

Precision is 0.78 %


In [55]:
print("Recall is {:.2f} %".format(recall))

Recall is 1.00 %


In [56]:
print("F1Score is {:.3f} %".format(F1Score))

F1Score is 0.874 %


#  Training naive bayes using bigram

In [57]:
vectorizer,counts,test_cv,test_counts =  creating_ngram_model(corpus,test_corpus,2)


In [58]:
spamCount,hamCount = counter(train)
spamFilter, hamFilter = create_matixes(train,counts)
spamOccurence ,hamOccurence = ham_spam_occurence(spamFilter, hamFilter)
features = vectorizer.get_feature_names()
occurenceDf = occurence(spamOccurence,hamOccurence,features)

ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.

In [ ]:
occurenceDf.sort_values('spam').tail(10)

In [ ]:
occurenceDf.sort_values('diff').tail(10)

In [ ]:
occurenceDf.sort_values('ham').tail(10)

In [ ]:
test_features = test_cv.get_feature_names()
test_rows = test_counts.toarray()
totalSpam = occurenceDf['spam'].sum()
totalHam = occurenceDf['ham'].sum()

In [ ]:
TP, FP, FN, TN = naive_bayes(occurenceDf, spamCount,hamCount,test_features,test_rows, totalSpam,totalHam)

In [ ]:
accuracy = (TP + TN) / (TP +  FP+ TN + FN)
precision = TP /(TP +FP)
recall = TP /(TP +FN)
F1Score = (2*(recall* precision))/(precision  +recall )

In [ ]:
print("Accuracy is {:.2f} %".format(accuracy))

In [ ]:
print("Precision is {:.2f} %".format(precision))

In [ ]:
print("Recall is {:.2f} %".format(recall))

In [ ]:
print("F1Score is {:.2f} %".format(F1Score))

# How algorithm works

1. Read files using pandas library
2. Eliminate stop words using stopwords.words('english')
3. Splitting data to train and test
4. Get corpus as python dictionary 
5. Count all spams and hams in the corpus
6. Create  2 numpy matrices for spams and hams to use them in naive_bayes algorithm
7. For each text in test dataset:
8.  Get all words 
9. Find probility overall spamLogProb and hamLogProb
10. Find the unigram prob for each ones in each text row
11. If the prob of ham bigger than prob of spam, then it's ham
12. Else it's spam
13. Cacluate TP, FP, FN, TN
14. Repeat without stop words
15. Repeat all using bigram


# Conclution 

1. it's better to use unigram than bigram or tigram since it's faster and accuracy is better
2. Time complexity increased as long as we have more grams and that's not practical for production
3. Words such as mail,get, free, money, business, information are the most occurance words in spam corpus 
4. Words such as would, please,  com, subjects are most occurance words in ham corpus
5. Stop words does not affect accuracy but it affect most occurance words in the list as we observed
6. Using bigram 